In [121]:
#import required packages
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [122]:
#read the data
df = pd.read_csv("DailyData.csv", parse_dates=['date'], delimiter=',')

In [123]:
#check the dtypes
print(df.dtypes)

campaign name                           object
date                            datetime64[ns]
clicks                                   int64
impressions                              int64
cost_micros                              int64
bidding_strategy_type                   object
conversions                            float64
cost_per_conversion                    float64
Click-through rate (CTR)               float64
average cost per click (CPC)           float64
cost-per-acquisition (CPA)             float64
Cost                                   float64
conversion per euro                    float64
dtype: object


In [124]:
df['Date_Time'] = pd.to_datetime(df.date , format = '%d/%m/%Y %H.%M.%S')
data = df.drop(['date'], axis=1)
data.index = data.Date_Time

In [125]:
data.head(3)

,campaign name,clicks,impressions,cost_micros,bidding_strategy_type,conversions,cost_per_conversion,Click-through rate (CTR),average cost per click (CPC),cost-per-acquisition (CPA),Cost,conversion per euro,Date_Time
Date_Time,,,,,,,,,,,,,
2023-11-28,SHOPTIMISED PMax-46 - all products-M,0,33,0,MAXIMIZE_CONVERSION_VALUE,0.0,NaN,0.000000,0.000000,0.000000,0.00,0.000000,2023-11-28
2023-11-29,SHOPTIMISED PMax-46 - all products-M,11,1139,4060000,MAXIMIZE_CONVERSION_VALUE,1.0,4060000.0,0.965759,0.369091,4.060000,4.06,0.246305,2023-11-29
2023-11-30,SHOPTIMISED PMax-46 - all products-M,7,655,1430000,MAXIMIZE_CONVERSION_VALUE,0.0,NaN,1.068702,0.204286,4.085714,1.43,0.000000,2023-11-30


In [126]:
data.drop(['campaign name','cost_micros','bidding_strategy_type','cost_per_conversion','Date_Time'],axis=1, inplace=True)

In [127]:
#creating the train and validation set
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

In [128]:
train.dtypes

clicks                            int64
impressions                       int64
conversions                     float64
Click-through rate (CTR)        float64
average cost per click (CPC)    float64
cost-per-acquisition (CPA)      float64
Cost                            float64
conversion per euro             float64
dtype: object

In [129]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)

# Determine the optimal lag order
lag_order = model.select_order()
print(lag_order.summary())

# Fit the model with the optimal lag order
optimal_lag = lag_order.selected_orders['HQIC']  # You can choose 'aic', 'bic', 'hqic', etc.
model_fit = model.fit(optimal_lag)

# Summary of the fitted model
print(model_fit.summary())

 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        12.60      12.78*   2.962e+05       12.67
1        11.81       13.42   1.347e+05      12.46*
2        11.24       14.28  7.679e+04*       12.47
3        11.39       15.87   9.204e+04       13.21
4        11.33       17.24   9.193e+04       13.73
5        11.12       18.47   8.187e+04       14.11
6        11.20       19.98   1.022e+05       14.76
7        11.59       21.80   1.876e+05       15.74
8        11.25       22.90   1.807e+05       15.98
9        10.84       23.92   1.833e+05       16.16
10       10.54       25.05   2.442e+05       16.44
11       10.03       25.97   3.334e+05       16.51
12       8.649       26.03   2.717e+05       15.71
13      6.362*       25.17   1.593e+05       14.00
--------------------------------------------------


C:\Users\Digitweb Sathurjan\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


KeyError: 'HQIC'

In [130]:
# make prediction on validation
# Make prediction on the validation set
n_steps = len(valid)
last_obs = train.values[-optimal_lag:]
prediction = model_fit.forecast(last_obs,steps=n_steps)

In [131]:
# Convert predictions to a DataFrame
prediction_df = pd.DataFrame(prediction, index=valid.index, columns=train.columns)
prediction_df

,clicks,impressions,conversions,Click-through rate (CTR),average cost per click (CPC),cost-per-acquisition (CPA),Cost,conversion per euro
Date_Time,,,,,,,,
2024-04-16,156.836952,14327.941456,7.114354,1.098279,0.296546,7.694391,45.397518,0.157324
2024-04-17,149.802912,13823.641759,6.551429,1.090159,0.292285,7.895046,43.325420,0.158047
2024-04-18,143.695992,13310.753414,6.282363,1.083762,0.291749,7.825329,41.597090,0.160656
2024-04-19,138.644621,12872.034355,6.087663,1.079790,0.291220,7.777625,40.107797,0.163279
2024-04-20,134.488582,12509.685866,5.928734,1.076742,0.290663,7.739998,38.863696,0.165463
2024-04-21,131.071555,12211.715470,5.797701,1.074250,0.290175,7.708553,37.836819,0.167244
2024-04-22,128.262852,11966.781707,5.689877,1.072201,0.289768,7.682518,36.991995,0.168702
2024-04-23,125.954433,11765.467172,5.601236,1.070518,0.289433,7.661095,36.297504,0.169898
2024-04-24,124.057262,11600.013838,5.528383,1.069135,0.289158,7.643492,35.726709,0.170880


In [132]:
#Combine the actual result and predicted result side by side
result=pd.DataFrame()
result['Actual Clicks']=valid.clicks
result['Predicted Clicks']=prediction_df.clicks

result['Actual impressions']=valid.impressions
result['Predicted impressions']=prediction_df.impressions

result['Actual conversions']=valid.conversions
result['Predicted conversions']=prediction_df.conversions

result['Actual Cost']=valid.Cost
result['Predicted Cost']=prediction_df.Cost

result['Actual Click-through rate (CTR)']=valid['Click-through rate (CTR)']
result['Predicted Click-through rate (CTR)']=prediction_df['Click-through rate (CTR)']

result['Actual average cost per click (CPC)']=valid['average cost per click (CPC)']
result['Predicted average cost per click (CPC)']=prediction_df['average cost per click (CPC)']

result['Actual cost-per-acquisition (CPA)']=valid['cost-per-acquisition (CPA)']
result['Predicted cost-per-acquisition (CPA)']=prediction_df['cost-per-acquisition (CPA)']

result['Actual conversion per euro']=valid['conversion per euro']
result['Predicted conversion per euro']=prediction_df['conversion per euro']

In [136]:
result.head(30)

,Actual Clicks,Predicted Clicks,Actual impressions,Predicted impressions,Actual conversions,Predicted conversions,Actual Cost,Predicted Cost,Actual Click-through rate (CTR),Predicted Click-through rate (CTR),Actual average cost per click (CPC),Predicted average cost per click (CPC),Actual cost-per-acquisition (CPA),Predicted cost-per-acquisition (CPA),Actual conversion per euro,Predicted conversion per euro
Date_Time,,,,,,,,,,,,,,,,
2024-04-16,146,156.836952,11636,14327.941456,10.079527,7.114354,54.800000,45.397518,1.254727,1.098279,0.375342,0.296546,5.436763,7.694391,0.183933,0.157324
2024-04-17,123,149.802912,11443,13823.641759,4.868349,6.551429,38.325701,43.325420,1.074893,1.090159,0.311591,0.292285,7.872423,7.895046,0.127026,0.158047
2024-04-18,190,143.695992,17217,13310.753414,9.134768,6.282363,56.074287,41.597090,1.103560,1.083762,0.295128,0.291749,6.138556,7.825329,0.162905,0.160656
2024-04-19,175,138.644621,15549,12872.034355,6.891433,6.087663,48.527914,40.107797,1.125474,1.079790,0.277302,0.291220,7.041774,7.777625,0.142010,0.163279
2024-04-20,170,134.488582,15435,12509.685866,6.016440,5.928734,54.936154,38.863696,1.101393,1.076742,0.323154,0.290663,9.131007,7.739998,0.109517,0.165463
2024-04-21,167,131.071555,14792,12211.715470,5.166667,5.797701,53.621576,37.836819,1.128989,1.074250,0.321087,0.290175,10.378369,7.708553,0.096354,0.167244
2024-04-22,167,128.262852,15258,11966.781707,4.321665,5.689877,50.848553,36.991995,1.094508,1.072201,0.304482,0.289768,11.765964,7.682518,0.084991,0.168702
2024-04-23,156,125.954433,14744,11765.467172,6.358181,5.601236,50.670000,36.297504,1.058058,1.070518,0.324808,0.289433,7.969260,7.661095,0.125482,0.169898
2024-04-24,128,124.057262,13416,11600.013838,4.522599,5.528383,30.221460,35.726709,0.954085,1.069135,0.236105,0.289158,6.682321,7.643492,0.149649,0.170880


In [137]:
#check rmse
cols = train.columns
from math import sqrt
from sklearn.metrics import mean_squared_error

for i in cols:
    print('rmse value for', i, 'is : ', sqrt(mean_squared_error(prediction_df[i], valid[i])))

rmse value for clicks is :  47.001072038654925
rmse value for impressions is :  4283.773575681952
rmse value for conversions is :  2.64165124528722
rmse value for Click-through rate (CTR) is :  0.12650445033668245
rmse value for average cost per click (CPC) is :  0.08169620262761304
rmse value for cost-per-acquisition (CPA) is :  7.677515349666301
rmse value for Cost is :  12.305461751340538
rmse value for conversion per euro is :  0.08325966036479787


In [138]:
#make final predictions
last_obs = data.values[-optimal_lag:]
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(last_obs, steps=10)
print(yhat)

[[6.90402555e+01 6.68059440e+03 3.83802504e+00 1.04249138e+00
  2.91549918e-01 7.92714579e+00 1.98124556e+01 2.01152714e-01]
 [7.69083577e+01 7.48790903e+03 3.75597616e+00 1.03626190e+00
  2.84419592e-01 7.42055634e+00 2.16509531e+01 1.94411471e-01]
 [8.36404258e+01 8.07905159e+03 3.97925211e+00 1.03968181e+00
  2.83832613e-01 7.32300432e+00 2.36016953e+01 1.91353935e-01]
 [8.92480961e+01 8.56563318e+03 4.19191475e+00 1.04370299e+00
  2.84280846e-01 7.33537816e+00 2.52662473e+01 1.88736136e-01]
 [9.38812634e+01 8.96869560e+03 4.36969494e+00 1.04711711e+00
  2.84835132e-01 7.36799917e+00 2.66518093e+01 1.86449443e-01]
 [9.76956577e+01 9.30108359e+03 4.51611132e+00 1.04991094e+00
  2.85349561e-01 7.40033061e+00 2.77965813e+01 1.84510399e-01]
 [1.00832331e+02 9.57456696e+03 4.63653026e+00 1.05220088e+00
  2.85792426e-01 7.42848559e+00 2.87394085e+01 1.82897247e-01]
 [1.03410715e+02 9.79941157e+03 4.73553116e+00 1.05408144e+00
  2.86162863e-01 7.45211305e+00 2.95148916e+01 1.81565679e-01]


C:\Users\Digitweb Sathurjan\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [139]:
#convert the final result to a dataframe
resultFrame=pd.DataFrame(yhat,columns=data.columns)

In [140]:
resultFrame

,clicks,impressions,conversions,Click-through rate (CTR),average cost per click (CPC),cost-per-acquisition (CPA),Cost,conversion per euro
0,69.040256,6680.594398,3.838025,1.042491,0.291550,7.927146,19.812456,0.201153
1,76.908358,7487.909028,3.755976,1.036262,0.284420,7.420556,21.650953,0.194411
2,83.640426,8079.051588,3.979252,1.039682,0.283833,7.323004,23.601695,0.191354
3,89.248096,8565.633183,4.191915,1.043703,0.284281,7.335378,25.266247,0.188736
4,93.881263,8968.695604,4.369695,1.047117,0.284835,7.367999,26.651809,0.186449
5,97.695658,9301.083592,4.516111,1.049911,0.285350,7.400331,27.796581,0.184510
6,100.832331,9574.566955,4.636530,1.052201,0.285792,7.428486,28.739408,0.182897
7,103.410715,9799.411573,4.735531,1.054081,0.286163,7.452113,29.514892,0.181566
8,105.529894,9984.222033,4.816907,1.055627,0.286469,7.471682,30.152407,0.180470
9,107.271566,10136.113682,4.883788,1.056897,0.286722,7.487810,30.676399,0.179568


In [141]:
# Generate date range for the forecasted values
date_range = pd.date_range(start=data.index[-1], periods=11)
resultFrame.index = date_range[1:]

In [142]:
resultFrame

,clicks,impressions,conversions,Click-through rate (CTR),average cost per click (CPC),cost-per-acquisition (CPA),Cost,conversion per euro
2024-05-22,69.040256,6680.594398,3.838025,1.042491,0.291550,7.927146,19.812456,0.201153
2024-05-23,76.908358,7487.909028,3.755976,1.036262,0.284420,7.420556,21.650953,0.194411
2024-05-24,83.640426,8079.051588,3.979252,1.039682,0.283833,7.323004,23.601695,0.191354
2024-05-25,89.248096,8565.633183,4.191915,1.043703,0.284281,7.335378,25.266247,0.188736
2024-05-26,93.881263,8968.695604,4.369695,1.047117,0.284835,7.367999,26.651809,0.186449
2024-05-27,97.695658,9301.083592,4.516111,1.049911,0.285350,7.400331,27.796581,0.184510
2024-05-28,100.832331,9574.566955,4.636530,1.052201,0.285792,7.428486,28.739408,0.182897
2024-05-29,103.410715,9799.411573,4.735531,1.054081,0.286163,7.452113,29.514892,0.181566
2024-05-30,105.529894,9984.222033,4.816907,1.055627,0.286469,7.471682,30.152407,0.180470
2024-05-31,107.271566,10136.113682,4.883788,1.056897,0.286722,7.487810,30.676399,0.179568
